In [1]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append(r'C:\Users\TristanDelforge\Documents\repos\sports_predictor')
sys.path.append(r'C:\Users\TristanDelforge\Documents\repos\sports_predictor\nba')



base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
from process_data import DataManager

In [3]:
dm = DataManager(testing = 2000)
dm.update_raw_datasets()
# dm.build_timeseries(4, False)
# dm.combine_timeseries(4, False)


function: 'create_feature_target_df' starting
function: 'create_feature_target_df' finished in  0.00 seconds
function: 'calculate_team_game_rating' starting


2000it [00:10, 185.95it/s]


function: 'calculate_team_game_rating' finished in  10.78 seconds
function: 'calculate_team_game_rating' starting


2000it [00:10, 188.68it/s]


function: 'calculate_team_game_rating' finished in  10.62 seconds
function: 'assign_date_since_last_game' starting
function: 'assign_date_since_last_game' finished in  0.03 seconds
function: 'assign_home_for_teams' starting
function: 'find_team_home_loc' starting
function: 'find_team_home_loc' finished in  0.43 seconds
function: 'assign_home_for_teams' finished in  0.53 seconds
function: 'build_moving_average_features' starting


...

function: 'build_moving_average_features' finished in  31.87 seconds
function: 'build_moving_average_features' starting


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:39<00:00,  1.31s/it]


function: 'build_moving_average_features' finished in  39.38 seconds
function: 'build_moving_average_features' starting


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:29<00:00,  1.01it/s]


function: 'build_moving_average_features' finished in  29.67 seconds
function: 'build_event_features' starting
build_event_features start: 87270
build_event_features end: 0
function: 'build_event_features' finished in  24.81 seconds


In [4]:
x1, x2, y, cols = dm.get_labeled_data(4, False, False)
print(x1.shape, x2.shape, y.shape, len(cols))

function: 'get_labeled_data' starting


self.feature_df columns: ['team_tag', 'opponent_tag', 'game_key', 'date_str', 'win', 'feature_team_pregame_rating_0', 'feature_team_pregame_rating_1', 'dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6', 'year_2010', 'year_2011', 'month_1', 'month_2', 'month_3', 'month_11', 'month_12', 'days_since_last_match', 'feature_home', 'feature_team_rl_trend_ft_pct_3', 'feature_team_rl_avg_drb_pct_3', 'feature_team_rl_trend_ast_pct_3', 'feature_team_rl_avg_fg3_3', 'feature_team_rl_avg_mp_3', 'feature_team_rl_avg_fg_pct_3', 'feature_team_rl_trend_mp_3', 'feature_team_rl_trend_off_rtg_3', 'feature_team_rl_avg_ts_pct_3', 'feature_team_rl_trend_blk_pct_3', 'feature_team_rl_avg_blk_pct_3', 'feature_team_rl_trend_fg3a_3', 'feature_team_rl_trend_fg3_3', 'feature_team_rl_trend_efg_pct_3', 'feature_team_rl_avg_tov_pct_3', 'feature_team_rl_avg_blk_3', 'feature_team_rl_trend_blk_3', 'feature_team_rl_avg_ft_pct_3', 'feature_team_rl_avg_ast_pct_3', 'feature_team_rl_avg_orb_pct_3', 'feature_team_rl_

function: 'get_labeled_data' finished in  0.26 seconds
(0,) (0,) (0,) 429


In [5]:
np.isnan(x1).astype(int).sum()

0

In [6]:
x2_df = pd.DataFrame(data=x2,
                    columns = cols)
x2_df.isna().sum().sum()

ValueError: Empty data passed with indices specified.

In [ ]:
x2_df.head(10)

In [ ]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)

team_data.shape, player_data.shape

In [ ]:
team_data.head()

In [ ]:
player_data.head()

- Negatively correlated with win
- High stat def rating is highly correlated with losing (.51). Teams under pressure don't win that game.
- Personal fouls also correlated with losing (.13)
- Turnovers - (investigate, not intuitive)
- 

In [ ]:
team_data_corr = team_data.corr()
team_data_corr = team_data_corr[(team_data_corr['win'] > .2)|(team_data_corr['win'] < -.08)]
team_data_corr.sort_values('win')[['win']]

In [ ]:
player_data_corr = player_data.corr()
player_data_corr = player_data_corr[(player_data_corr['win'] > .05)|(player_data_corr['win'] < -.05)]
player_data_corr.sort_values('win')[['win']]

In [ ]:

past_n_game_obj = None
with open('{data_path}/{db_name}.pkl'.format(data_path=data_path,
                                                             db_name='past_n_game_dataset_32_False'), 'rb') as f:
    past_n_game_dataset = pickle.load(f)
past_n_game_dataset.keys()


In [ ]:
team_data[(team_data['team_tag'] == 'pho')&(team_data['month'] == 12)&(team_data['year'] == 2017)][['month', 'day', 'ast']].head(20)

In [ ]:
initial_team_data_columns = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                          'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                          'ft_pct',
                                          'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                          'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                          'ts_pct',
                                          'usg_pct', 'home', 'r1', 'r2', 'r3', 'r4']

pd.DataFrame(data=past_n_game_dataset['pho']["['2017-12-31', 'phi', 'pho']"])


In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results.sort_values('accuracy', ascending = False).head(10)


In [ ]:
results.shape

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results.sort_values('accuracy', ascending = False).head(10)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 4].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 8].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 16].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 32].sort_values('accuracy', ascending = False).head(20)